In [70]:
import pandas as pd
import numpy as np
import os
import nsepy
import wget
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from datetime import datetime, timedelta

df2016 = pd.read_csv("nifty2016.csv")
df2017 = pd.read_csv("nifty2017.csv")
df2018 = pd.read_csv("nifty2018.csv")
df2019 = pd.read_csv("nifty2019.csv")
df2020 = pd.read_csv("nifty2020.csv")
df2021 = pd.read_csv("nifty2021.csv")
dfin = df2016.append(df2017)
dfin = dfin.append(df2018)
dfin = dfin.append(df2019)
dfin = dfin.append(df2020)
dfin = dfin.append(df2021)
# dfin.drop(['Shares Traded','Close','Open','Turnover (Rs. Cr)'],inplace=True,axis=1)
# dfin['Close'] = (dfin['Low'] + dfin['High']) / 2
# dfin.drop(['Low','High'],inplace=True,axis=1)
# dfin['Gap'] = dfin['Open'] - dfin['Close'].shift(1)
# dfin['Gap'] = dfin['Gap'].fillna(0)
dfin.drop(['Shares Traded','Open','Low','High','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin.head()

,Date,Close
0,01-Jan-2016,7963.20
1,04-Jan-2016,7791.30
2,05-Jan-2016,7784.65
3,06-Jan-2016,7741.00
4,07-Jan-2016,7568.30


In [71]:
option_file = 'OptionPrice_dumpall.csv'

def load_option_prices():
    if os.path.exists(option_file):
        df = pd.read_csv(option_file)
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        df['ExpiryDay'] = pd.to_datetime(df['ExpiryDay'], format='%Y-%m-%d')
    else:
        df = pd.DataFrame(columns=['Date', 'Name', 'Type', 'StrikePrice', 'ExpiryDay','Price'])
    return df  
options_df = load_option_prices()


def get_url(dt):
    base_url = "http://www1.nseindia.com/content/historical/DERIVATIVES/" #2020/FEB/fo06FEB2020bhav.csv.zip
    yr = dt.year
    day = dt.day
    mon = dt.strftime('%b').upper()
    filename = 'fo{2:02d}{1}{0}bhav.csv.zip'.format(yr,mon,day)
    return '{0}{1}/{2}/{3}'.format(base_url,yr,mon,filename), filename

In [72]:
options_df.tail()

,Date,Name,Type,StrikePrice,ExpiryDay,Price,ClosePrice,LowPrice,HighPrice,OpenPrice
6180,2021-12-20,NIFTY,CE,17600,2021-12-30,NaN,11.35,10.00,30.00,30.00
6181,2021-12-21,NIFTY,CE,17600,2021-12-30,NaN,8.25,7.70,16.50,13.00
6182,2021-12-22,NIFTY,CE,17600,2021-12-30,NaN,10.35,7.45,11.80,9.05
6183,2021-12-23,NIFTY,CE,17600,2021-12-30,NaN,10.25,9.15,16.95,13.95
6184,2021-12-24,NIFTY,CE,17600,2021-12-30,NaN,4.55,4.20,13.95,11.00


In [73]:
df = dfin.copy()
df['Date'] = df['Date'].apply(lambda x : datetime.strptime(x,"%d-%b-%Y"))
df['Day'] = df['Date'].apply(lambda x: x.weekday())
df['Month'] = df['Date'].apply(lambda x: x.month)
df['Year'] = df['Date'].apply(lambda x: x.year)
df['MonthYear'] = df['Date'].apply(lambda x: str(x.year) + str(x.month))
df.head()

,Date,Close,Day,Month,Year,MonthYear
0,2016-01-01,7963.20,4,1,2016,20161
1,2016-01-04,7791.30,0,1,2016,20161
2,2016-01-05,7784.65,1,1,2016,20161
3,2016-01-06,7741.00,2,1,2016,20161
4,2016-01-07,7568.30,3,1,2016,20161


In [74]:
OTM = 200
call_sell_dates = ['22-01-2021','17-02-2021','12-03-2021','19-10-2021','16-11-2021','13-12-2021',
                  '20-01-2020','24-06-2020','01-09-2020','15-10-2020','25-11-2020',
                  '08-02-2019','22-03-2019','03-04-2019','07-05-2019','23-05-2019','05-07-2019']
call_sell_dates = [datetime.strptime(i,"%d-%m-%Y") for i in call_sell_dates]
dfsellcall = df[df.Date.isin(call_sell_dates)].copy()
dfsellcall.rename(columns = {'Date':'Date_Sell', 'Close':'Close_Sell'}, inplace = True)
dfsellcall.head()

,Date_Sell,Close_Sell,Day,Month,Year,MonthYear
28,2019-02-08,10943.60,4,2,2019,20192
56,2019-03-22,11456.90,4,3,2019,20193
64,2019-04-03,11643.95,2,4,2019,20194
84,2019-05-07,11497.90,1,5,2019,20195
96,2019-05-23,11657.05,3,5,2019,20195


In [75]:
dfsellcall['StrikePrice'] = dfsellcall['Close_Sell'].apply(lambda x: round(x + OTM,-2))
dfsellcall.tail()

,Date_Sell,Close_Sell,Day,Month,Year,MonthYear,StrikePrice
32,2021-02-17,15208.90,2,2,2021,20212,15400.0
48,2021-03-12,15030.95,4,3,2021,20213,15200.0
196,2021-10-19,18418.75,1,10,2021,202110,18600.0
215,2021-11-16,17999.20,1,11,2021,202111,18200.0
233,2021-12-13,17368.25,0,12,2021,202112,17600.0


In [76]:
def options_get_history(row, options_df):
    index_option = "NIFTY"
    call_strike_price = int(row['StrikePrice'])
    callopenprice, callhighprice, calllowprice, callcloseprice = 0,0,0,0
    found_price_call = options_df.loc[(options_df['Date'] == row['Date'].strftime("%Y-%m-%d")) & (options_df['Name'] == index_option) 
                                     & (options_df['Type'] == 'CE') & (options_df['StrikePrice'] == call_strike_price) 
                                     & (options_df['ExpiryDay'] == row['ExpiryDay'].strftime("%Y-%m-%d"))]
    if len(found_price_call) == 0: 
        
        #http://www1.nseindia.com/content/historical/DERIVATIVES/2020/FEB/fo06FEB2020bhav.csv.zip
        url, filename = get_url(row['Date'])
        print('Fetching Price from NSE dumps {0}_{1}_CE on {2}. FileName: {3}'.format(row['ExpiryDay'], call_strike_price, row['Date'].strftime("%Y-%m-%d"),url))
        op_file = os.path.join('downloads',filename)
        if not os.path.exists(op_file):
            wget.download(url, out='./downloads')
            print()
        df = pd.read_csv(op_file)
        df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'], format='%d-%b-%Y')
        df = df[df['INSTRUMENT'] == 'OPTIDX']
        df = df[df['SYMBOL'] == index_option]
        df = df[df['STRIKE_PR'] == call_strike_price]
        df = df[df['EXPIRY_DT'] == row['ExpiryDay'].strftime("%Y-%m-%d")]
        callopenprice = df[df['OPTION_TYP'] == 'CE']['OPEN'].values[0]
        callhighprice = df[df['OPTION_TYP'] == 'CE']['HIGH'].values[0]
        calllowprice = df[df['OPTION_TYP'] == 'CE']['LOW'].values[0]
        callcloseprice = df[df['OPTION_TYP'] == 'CE']['CLOSE'].values[0]
        temp_dict = {}
        temp_dict[row['Date']] = {'Name'          : index_option, 
                                    'Type'        : 'CE', 
                                    'StrikePrice' : call_strike_price,
                                    'ExpiryDay'   : row['ExpiryDay'],
                                    'ClosePrice'  : callcloseprice,
                                    'LowPrice'    : calllowprice,
                                    'HighPrice'   : callhighprice,
                                    'OpenPrice'   : callopenprice,
                                    }
        temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
        temp_df['Date'] = temp_df.index
        temp_df.reset_index(inplace=True,drop=True)
        options_df = options_df.append(temp_df, ignore_index = True,sort=False)
        
        temp_df = None 
    else:
        callopenprice = found_price_call['OpenPrice'].values[0]
        callhighprice = found_price_call['HighPrice'].values[0]
        calllowprice = found_price_call['LowPrice'].values[0]
        callcloseprice = found_price_call['ClosePrice'].values[0]
    return callopenprice, callhighprice, calllowprice, callcloseprice, options_df

In [77]:
dfsellcall.tail()

,Date_Sell,Close_Sell,Day,Month,Year,MonthYear,StrikePrice
32,2021-02-17,15208.90,2,2,2021,20212,15400.0
48,2021-03-12,15030.95,4,3,2021,20213,15200.0
196,2021-10-19,18418.75,1,10,2021,202110,18600.0
215,2021-11-16,17999.20,1,11,2021,202111,18200.0
233,2021-12-13,17368.25,0,12,2021,202112,17600.0


In [78]:
transcallsellprice, transcallhigh, transcalllow, transcallbuyprice, transcallhighdate, transcalllowdate = [],[],[],[],[],[]
for i, r in dfsellcall.iterrows():
    expiry = list(sorted(get_expiry_date(year=r['Year'], month=r['Date_Sell'].month, stock=True)))[-1]
    exp_datetime = datetime(expiry.year,expiry.month,expiry.day)
    dfw = df[(df["Date"] >= r['Date_Sell']) & (df["Date"] <= exp_datetime)][['Date','Close']]
    row = {
           "Date": r['Date_Sell'], 
           "Name": "NIFTY",
           "StrikePrice": r['StrikePrice'],
           "ExpiryDay": expiry
          }
    callopen, callhigh, calllow, callclose, call_date = [], [], [], [], []
    for ii, ir in dfw.iterrows():
        row['Date'] = ir['Date']
        callopenprice, callhighprice, calllowprice, callcloseprice, options_df = options_get_history(row, options_df)
        callopen.append(callopenprice)
        callhigh.append(callhighprice)
        calllow.append(calllowprice)
        callclose.append(callcloseprice)
        call_date.append(row['Date'])
#         print(callhighprice, calllowprice, callcloseprice)

    transcallsellprice.append(callclose[0])
    transcallbuyprice.append(callclose[-1])
    
    maxi = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(callhigh[1:],call_date[1:]) if i >= callclose[0] * 1.5]
    if maxi:
        max_call, max_date = maxi[0]
    else:
        max_call, max_date= max(callhigh[1:]),0
    transcallhigh.append(max_call)
    transcallhighdate.append(max_date)
    lowe = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(calllow[1:],call_date[1:]) if i <= callclose[0] * 0.5]
    if lowe:
        low_call, low_date = lowe[-1]
    else:
        low_call, low_date= max(calllow[1:]), 0
    transcalllow.append(low_call)
    transcalllowdate.append(low_date)

#     print()
#     print(transcallsellprice, transcallhigh, transcalllow, transcallbuyprice)
#     break
dfsellcall['transcallsellprice'] = transcallsellprice
dfsellcall['transcallhigh'] = transcallhigh
dfsellcall['transcallhighdate'] = transcallhighdate
dfsellcall['transcalllow'] = transcalllow
dfsellcall['transcalllowdate'] = transcalllowdate
dfsellcall['transcallbuyprice'] = transcallbuyprice
dfsellcall.head()

Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-08. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo08FEB2019bhav.csv.zip
Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-11. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo11FEB2019bhav.csv.zip
Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-12. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo12FEB2019bhav.csv.zip
Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-13. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo13FEB2019bhav.csv.zip
Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-14. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo14FEB2019bhav.csv.zip
Fetching Price from NSE dumps 2019-02-28_11100_CE on 2019-02-15. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/FEB/fo15FEB2019bhav.csv.zip
Fetching Price f

Fetching Price from NSE dumps 2019-05-30_11700_CE on 2019-05-28. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo28MAY2019bhav.csv.zip
Fetching Price from NSE dumps 2019-05-30_11700_CE on 2019-05-29. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo29MAY2019bhav.csv.zip
Fetching Price from NSE dumps 2019-05-30_11700_CE on 2019-05-30. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo30MAY2019bhav.csv.zip
Fetching Price from NSE dumps 2019-05-30_11900_CE on 2019-05-23. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo23MAY2019bhav.csv.zip
Fetching Price from NSE dumps 2019-05-30_11900_CE on 2019-05-24. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo24MAY2019bhav.csv.zip
Fetching Price from NSE dumps 2019-05-30_11900_CE on 2019-05-27. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2019/MAY/fo27MAY2019bhav.csv.zip
Fetching Price f

Fetching Price from NSE dumps 2020-09-24_11700_CE on 2020-09-22. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/SEP/fo22SEP2020bhav.csv.zip
Fetching Price from NSE dumps 2020-09-24_11700_CE on 2020-09-23. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/SEP/fo23SEP2020bhav.csv.zip
Fetching Price from NSE dumps 2020-09-24_11700_CE on 2020-09-24. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/SEP/fo24SEP2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_11900_CE on 2020-10-15. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo15OCT2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_11900_CE on 2020-10-16. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo16OCT2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_11900_CE on 2020-10-19. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo19OCT2020bhav.csv.zip
Fetching Price f

Fetching Price from NSE dumps 2021-11-25_18200_CE on 2021-11-24. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/NOV/fo24NOV2021bhav.csv.zip
Fetching Price from NSE dumps 2021-11-25_18200_CE on 2021-11-25. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/NOV/fo25NOV2021bhav.csv.zip
Fetching Price from NSE dumps 2021-12-30_17600_CE on 2021-12-13. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/DEC/fo13DEC2021bhav.csv.zip
Fetching Price from NSE dumps 2021-12-30_17600_CE on 2021-12-14. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/DEC/fo14DEC2021bhav.csv.zip
Fetching Price from NSE dumps 2021-12-30_17600_CE on 2021-12-15. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/DEC/fo15DEC2021bhav.csv.zip
Fetching Price from NSE dumps 2021-12-30_17600_CE on 2021-12-16. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/DEC/fo16DEC2021bhav.csv.zip
Fetching Price f

,Date_Sell,Close_Sell,Day,Month,Year,MonthYear,StrikePrice,transcallsellprice,transcallhigh,transcallhighdate,transcalllow,transcalllowdate,transcallbuyprice
28,2019-02-08,10943.60,4,2,2019,20192,11100.0,73.55,61.00,0,0.05,2019-02-28,0.05
56,2019-03-22,11456.90,4,3,2019,20193,11700.0,8.10,8.75,0,0.05,2019-03-28,0.10
64,2019-04-03,11643.95,2,4,2019,20194,11800.0,126.70,127.00,0,0.05,2019-04-25,0.10
84,2019-05-07,11497.90,1,5,2019,20195,11700.0,251.65,415.00,2019-05-23,83.30,2019-05-24,238.90
96,2019-05-23,11657.05,3,5,2019,20195,11900.0,59.25,116.60,2019-05-27,12.50,2019-05-30,37.05


In [79]:
dfsellcall['RiskPrice'] = dfsellcall['transcallsellprice'] * 1.5
dfsellcall['StopLossHit'] = dfsellcall.apply(lambda x: "Yes" if x['transcallhigh'] - x['RiskPrice'] > 0 else "No", axis=1)
dfsellcall['StopLossHitinDays'] = dfsellcall.apply(lambda x: (datetime.strptime(x["transcallhighdate"],"%Y-%m-%d") - x['Date_Sell']).days if x['StopLossHit'] == "Yes" else "0", axis=1) 
dfsellcall['Profit'] = dfsellcall.apply(lambda x: (x['transcallsellprice'] - x['transcallbuyprice']) * 50 if x['StopLossHit'] == 'No' else (x['transcallsellprice'] - x['RiskPrice']) * 50 , axis=1)
dfsellcall['SellMargin'] = dfsellcall['Close_Sell'] * 6
dfsellcall['ProfitPercentage'] = dfsellcall['Profit'] / dfsellcall['SellMargin']
dfsellcall['CumProfit'] = dfsellcall['Profit'].cumsum()
dfsellcall['CumSellMargin'] = dfsellcall['SellMargin'].cumsum()
dfsellcall['row_num'] = dfsellcall.index + 1
dfsellcall['CumSellMargin'] = dfsellcall['CumSellMargin']  / dfsellcall['row_num']
dfsellcall['CumProfitPercentage'] = dfsellcall['CumProfit'] / dfsellcall['CumSellMargin']

dfsellcall.head()

,Date_Sell,Close_Sell,Day,Month,Year,MonthYear,StrikePrice,transcallsellprice,transcallhigh,transcallhighdate,...,RiskPrice,StopLossHit,StopLossHitinDays,Profit,SellMargin,ProfitPercentage,CumProfit,CumSellMargin,row_num,CumProfitPercentage
28,2019-02-08,10943.60,4,2,2019,20192,11100.0,73.55,61.00,0,...,110.325,No,0,3675.00,65661.6,0.055969,3675.00,2264.193103,29,1.623095
56,2019-03-22,11456.90,4,3,2019,20193,11700.0,8.10,8.75,0,...,12.150,No,0,400.00,68741.4,0.005819,4075.00,2357.947368,57,1.728198
64,2019-04-03,11643.95,2,4,2019,20194,11800.0,126.70,127.00,0,...,190.050,No,0,6330.00,69863.7,0.090605,10405.00,3142.564615,65,3.310990
84,2019-05-07,11497.90,1,5,2019,20195,11700.0,251.65,415.00,2019-05-23,...,377.475,Yes,16,-6291.25,68987.4,-0.091194,4113.75,3214.754118,85,1.279647
96,2019-05-23,11657.05,3,5,2019,20195,11900.0,59.25,116.60,2019-05-27,...,88.875,Yes,4,-1481.25,69942.3,-0.021178,2632.50,3538.107216,97,0.744042


In [80]:
# dfsellput.drop(['Gap','Gap_Open','Gap_Date','Open','Low','MonthYear','Year'],inplace=True,axis=1)
dfsellcall.to_csv("nifty_sellcall_optiondata_manual_OTM" + str(OTM) +".csv")

In [81]:
# options_df['ExpiryDay'] = options_df['ExpiryDay'].strftime("%Y-%m-%d")
options_df.drop_duplicates()
options_df.to_csv(option_file, header=True, sep=',', index=False)